# Question to be answered:

- Is the accuracy of model/human significantly better? In both force and mass questions?
- Is the distribution of responses significantly different?

In [1]:
import isaac.constants
isaac.constants.TQDM_DISABLE = True

from torch import nn
from torch.nn import Softmax
from isaac.utils import get_cuda_device_if_available
import joblib

from isaac.dataset import read_dataset, prepare_dataset
from isaac.models import MultiBranchModel, ComplexRNNModel
from isaac.constants import BASIC_TRAINING_COLS, MASS_CLASS_COLS, FORCE_CLASS_COLS
from isaac.evaluation import predict_with_a_group_of_saved_models, evaluate_saved_model
from isaac.statistical_tests import z_test

import torch
import glob
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
INTERVAL_SIZE = 1
FPS = 60
STEP_SIZE = 3
PD_STEP_SIZE = 10
SEQ_END = 2700

In [3]:
device = get_cuda_device_if_available()
print(device)

cuda:0


In [4]:
normalise_data = True
scaler_path = "scalers_js/passive_dual_scaler.sk"
network_dims = (len(BASIC_TRAINING_COLS), 25, 3, 0.5)
dataset_path = "../new_exp_data/exp7_passive.h5"
class_columns = [list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)]
multiclass = True

def get_question_predictions_for_group_of_models(question_type):    
    models = sorted(glob.glob("models/train_25_mb_with_js_data/best_"+question_type+"_model_seed_*.pt"))

    group_predictions = []
    
    predictions = predict_with_a_group_of_saved_models(tqdm(models), network_dims, dataset_path, 
                                                       training_columns=BASIC_TRAINING_COLS, 
                                                       class_columns=class_columns, step_size=STEP_SIZE, 
                                                       seq_end=SEQ_END, scaler_path=scaler_path,
                                                       arch=MultiBranchModel, multiclass=multiclass, trials=None,
                                                       predict_seq2seq=True)

    predictions = torch.stack(predictions)
    
    print(predictions.shape)
    if question_type == "mass":
        predictions = predictions[:, :, :, 0]
    else:
        predictions = predictions[:, :, :, 1]

    return predictions

def get_question_accuracy_for_group_of_models(question_type):    
    model_paths = tqdm(sorted(glob.glob("models/train_25_mb_with_js_data/best_"+question_type+"_model_seed_*.pt")))

    accuracies, predicted = evaluate_saved_model(model_paths, network_dims, dataset_path, 
                                                 training_columns=BASIC_TRAINING_COLS, class_columns=class_columns, 
                                                 step_size=STEP_SIZE, seq_end=SEQ_END, scaler_path=scaler_path,
                                                 arch=MultiBranchModel, multiclass=multiclass, trials=None)
    
    if question_type == "mass":
        question_index = 0
    else:
        question_index = 1

    accuracies = np.stack(accuracies)[:, question_index]
    
    predicted = [x[:, question_index].numpy() for x in predicted]

    return accuracies, predicted

def get_participant_accuracy(passive_responses, answer_column, question_type_answer):
    return [(df[answer_column] == df[question_type_answer]).sum() / len(df) 
            for _, df in passive_responses.groupby("cond_worldvar")]

def get_participant_accuracy_filtering_by_answer(passive_responses, answer_column, question_type_answer, filter_by_class):
    
    passive_responses = passive_responses.copy().query(question_type_answer+" == "+filter_by_class)
    
    return [(df[answer_column] == df[question_type_answer]).sum() / len(df) 
            for _, df in passive_responses.groupby("cond_worldvar")]

# T-test for MASS questions

## Load model's predictions

In [5]:
print("MASS")
question_type = "mass"
group_mass_seq_prediction = get_question_predictions_for_group_of_models(question_type)

print("\nFORCE")
question_type = "force"
group_force_seq_prediction = get_question_predictions_for_group_of_models(question_type)

  0%|          | 0/25 [00:00<?, ?it/s]

MASS


  0%|          | 0/25 [00:00<?, ?it/s]

torch.Size([25, 36, 45, 2, 3])

FORCE


100%|██████████| 25/25 [00:12<00:00,  2.12it/s]

torch.Size([25, 36, 45, 2, 3])


In [6]:
s = Softmax(dim=-1)
group_force_seq_prediction = s(group_force_seq_prediction)
group_mass_seq_prediction = s(group_mass_seq_prediction)

In [7]:
sim_second = [i for _ in range(25) for _ in range(36) for i in range(1, 46)]
model_seed = [i for i in range(25) for _ in range(36) for _ in range(1, 46)]
trial_number = [i for _ in range(25) for i in range(36) for _ in range(1, 46)]

In [15]:
mass_df = pd.DataFrame(data=group_mass_seq_prediction.reshape(40500, 3).numpy(), 
                       columns=["rnn_A", "rnn_B", "rnn_same"])
mass_df["sim_second"] = sim_second
mass_df["model_seed"] = model_seed
mass_df["trial_number"] = trial_number

In [11]:
print("MASS")
question_type = "mass"
mass_accuracies, group_mass_prediction = get_question_accuracy_for_group_of_models(question_type)

print("\nFORCE")
question_type = "force"
force_accuracies, group_force_prediction = get_question_accuracy_for_group_of_models(question_type)

  0%|          | 0/25 [00:00<?, ?it/s]

MASS


  0%|          | 0/25 [00:00<?, ?it/s]


FORCE


100%|██████████| 25/25 [00:12<00:00,  2.00it/s]


In [42]:
np.mean(mass_accuracies), np.mean(force_accuracies)

(50.88888888888889, 59.555555555555564)

In [12]:
from scipy.stats import mode

mode_mass_prediction = mode(np.stack(group_mass_prediction), axis=0)[0][0]
mode_force_prediction = mode(np.stack(group_force_prediction), axis=0)[0][0]

In [13]:
import matplotlib.pyplot as plt

In [45]:
def get_trial_dataframes(mass_predictions, force_predictions, mass_answers, force_answers):
    all_trials = []
        
    for (mass_trial_predictions, 
         force_trial_predictions, 
         mass_trial_answer, 
         force_trial_answer) in zip(mass_predictions, force_predictions, mass_answers, force_answers):
        
        rnn_class_columns = ["rnn_" + cl for cl in list(MASS_CLASS_COLS) + list(FORCE_CLASS_COLS)]
        all_predictions = torch.cat((mass_trial_predictions, force_trial_predictions), dim=1).numpy()
        trial_df = pd.DataFrame(all_predictions, columns=rnn_class_columns)
                
        trial_df["mode_mass_answer"] = np.repeat(MASS_CLASS_COLS[mass_trial_answer], repeats=trial_df.shape[0])
        trial_df["mode_force_answer"] = np.repeat(FORCE_CLASS_COLS[force_trial_answer], repeats=trial_df.shape[0])

        all_trials.append(trial_df)
    
    return all_trials

In [ ]:
all_trials = get_trial_dataframes(mean_seq_mass_prediction, mean_seq_force_prediction, mode_mass_prediction, mode_force_prediction)

In [16]:
import json

In [17]:
condition_world_variant = []
world_id = []

for condition_id in range(1, 5):
    filename = "../new_exp_data/physics_data%d.json" % condition_id
    fd = open(filename)
    sim_data = json.load(fd)
    
    for sim in sim_data:
        if sim["practice"]:
            continue
        condition_world_variant.append(sim["condition_world_variant"])
        world_id.append(sim["world_id"])

In [18]:
for trial, cwv, w_id in zip(all_trials, condition_world_variant, world_id):
    trial["condition_world_variant"] = cwv
    trial["world_id"] = w_id

NameError: name 'all_trials' is not defined

## Load human results

In [ ]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [ ]:
rdata_path = "../new_exp_data/e7_passive_io_rtheta.rdata"
r['load'](rdata_path)

responses = r["tw"].query("practice == 0")
responses.trial_type = responses.trial_type.astype(int)
responses.world_id = responses.world_id.astype(int)

In [ ]:
responses[['post_ent_mass.rtheta', 'post_ent_rel.rtheta']].head()

In [ ]:
force_answers = []
mass_answers = []

for trial in all_trials:
    this_trial_responses = responses[responses.world_id == trial.world_id.unique()[0]]
    
    assert this_trial_responses.true_mass.nunique() == 1
    assert this_trial_responses.true_rel.nunique() == 1
    
    force_answer = this_trial_responses.true_rel.unique()[0]
    trial["true_rel"] = force_answer
    mass_answer = this_trial_responses.true_mass.unique()[0]
    trial["true_mass"] = mass_answer
    
    force_columns = ["rnn_"+cl for cl in FORCE_CLASS_COLS]
    trial["is_rnn_force_correct"] = trial["mode_force_answer"].unique()[0] == force_answer
    mass_columns = ["rnn_"+cl for cl in MASS_CLASS_COLS]                          
    trial["is_rnn_mass_correct"] = trial["mode_mass_answer"].unique()[0] == mass_answer
    
    force_answers.append(trial["mode_force_answer"].unique()[0] == force_answer)
    mass_answers.append( trial["mode_mass_answer"].unique()[0] == mass_answer)
    
    trial["model_correct_force_answer_probability"] = trial["rnn_"+force_answer]
    trial["model_correct_mass_answer_probability"] = trial["rnn_"+mass_answer]

In [ ]:
np.mean(mass_answers), np.mean(force_answers)

In [ ]:
import glob
trials_pd_paths = sorted(glob.glob("../new_exp_data/pd_summaries/pd_summaries/cv*tt*.rdata"))

trial_pds = []

for path in trials_pd_paths:
    r['load'](path)
    trial = r['df.ext']
    trial.frame = np.arange(10, 2710, 10).tolist() * 4
    trial_pds.append(trial)    

In [ ]:
from scipy.stats import spearmanr

In [ ]:
correlation_dfs = []

for trial_pd, rnn_trial in zip(trial_pds, all_trials):
    
    window_size = int(FPS * INTERVAL_SIZE / PD_STEP_SIZE)
    pd_seq_end = SEQ_END // PD_STEP_SIZE
    
    pd_mass = trial_pd[["frame", "pd.mass.xyrtheta"]].groupby("frame").agg({"pd.mass.xyrtheta": np.mean})
    pd_mass = pd_mass["pd.mass.xyrtheta"].rolling(window=window_size).mean().values[window_size-1::window_size]

    pd_force = trial_pd[["frame", "pd.rel.xyrtheta"]].groupby("frame").agg({"pd.rel.xyrtheta": np.mean})
    pd_force = pd_force[:pd_seq_end]
    pd_force = pd_force["pd.rel.xyrtheta"].rolling(window=window_size).mean().values[window_size-1::window_size]
    
    rnn_mass_correct_answer_probability = rnn_trial["model_correct_mass_answer_probability"].values    
    rnn_force_correct_answer_probability = rnn_trial["model_correct_force_answer_probability"].values
    
    force_class = rnn_trial.true_rel.unique()[0]
    force_class_id = FORCE_CLASS_COLS.index(force_class)
    mass_class = rnn_trial.true_mass.unique()[0]
    mass_class_id = MASS_CLASS_COLS.index(mass_class)
    
    
    
    df = pd.DataFrame(columns=["second", "pd_mass", "pd_force", "mass_corr", "force_corr", "model_correct_mass_answer_probability", "model_correct_force_answer_probability"])
    df.pd_mass = pd_mass
    df.pd_force = pd_force
    df.true_rel = force_class
    df.true_mass = mass_class
    df.is_mass_correct = rnn_trial.mode_mass_answer.unique()[0] == df.true_mass
    df.is_force_correct = rnn_trial.mode_force_answer.unique()[0] == df.true_rel

    df["model_correct_mass_answer_probability"] = rnn_mass_correct_answer_probability
    df["model_correct_force_answer_probability"] = rnn_force_correct_answer_probability
    
    df.second = np.arange(INTERVAL_SIZE, 45+INTERVAL_SIZE, INTERVAL_SIZE)
    df.set_index("second", inplace=True)
        
    df.mass_corr = np.corrcoef(df.pd_mass, rnn_mass_correct_answer_probability)[0][1]
    df.force_corr = np.corrcoef(df.pd_force, rnn_force_correct_answer_probability)[0][1]
    
    correlation_dfs.append(df)

In [ ]:
corr_df = [[df.true_mass, df.mass_corr.unique()[0], df.is_mass_correct, 
            df.true_rel, df.force_corr.unique()[0], df.is_force_correct] for df in correlation_dfs]

corr_df = pd.DataFrame(data=corr_df, columns=["true_mass", "mass_corr", "is_mass_correct",
                                              "true_rel", "force_corr", "is_force_correct"])

In [ ]:
axes = corr_df.boxplot(column="mass_corr", by="true_mass", showfliers=False, figsize=(12, 6))
axes.get_figure().suptitle("")
plt.title("")
plt.ylabel("Correlation", fontsize=30, fontweight="bold")
plt.xlabel("Mass class", fontsize=30, fontweight="bold")
plt.tick_params(axis='both', labelsize=25)

axes.set_xticklabels(MASS_CLASS_COLS, fontweight='bold')
axes.set_yticks(np.arange(-10, 10, 2) / 10)
axes.set_yticklabels(np.arange(-10, 10, 2) / 10, fontweight='bold')

label = ""
for i in range(1, 4):    
    if i == 3:
        label = "RNN is correct"
    
    y = corr_df.mass_corr[(corr_df.true_mass == MASS_CLASS_COLS[i - 1]) & (corr_df.is_mass_correct)]
    x = np.random.normal(i, 0.055, size=len(y))
    plt.plot(x, y, 'b.', alpha=0.7, label=label, markersize=25)
    
    if i == 3:
        label = "RNN is wrong"
    
    y = corr_df.mass_corr[(corr_df.true_mass == MASS_CLASS_COLS[i - 1]) & (~corr_df.is_mass_correct)]
    x = np.random.normal(i, 0.055, size=len(y))
    plt.plot(x, y, 'rx', alpha=0.7, label=label, markersize=12, mew=5)

plt.legend(fontsize=25, prop={'size': 20, 'weight':'bold'})
plt.tight_layout()
# plt.savefig("cogsci_images/mass_corr_PD.pdf")
# plt.savefig("cogsci_images/mass_corr_PD.jpg")
    
    
axes = corr_df.boxplot(column="force_corr", by="true_rel", showfliers=False, figsize=(12, 6))
axes.get_figure().suptitle("")
plt.title("")
plt.ylabel("Correlation", fontsize=30, fontweight="bold")
plt.xlabel("Force class", fontsize=30, fontweight="bold")
plt.tick_params(axis='both', labelsize=25)

axes.set_xticklabels(FORCE_CLASS_COLS, fontweight='bold')
axes.set_yticks(np.arange(-10, 10, 2) / 10)
axes.set_yticklabels(np.arange(-10, 10, 2) / 10, fontweight='bold')


label = ""
for i in range(1, 4):    
    if i == 3:
        label = "RNN is correct"
    
    y = corr_df.force_corr[(corr_df.true_rel == FORCE_CLASS_COLS[i - 1]) & (corr_df.is_force_correct)]
    x = np.random.normal(i, 0.055, size=len(y))
    plt.plot(x, y, 'b.', alpha=0.7, label=label, markersize=25)
    
    if i == 3:
        label = "RNN is wrong"
    
    y = corr_df.force_corr[(corr_df.true_rel == FORCE_CLASS_COLS[i - 1]) & (~corr_df.is_force_correct)]
    x = np.random.normal(i, 0.055, size=len(y))
    plt.plot(x, y, 'rx', alpha=0.7, label=label, markersize=12, mew=5)

plt.legend(fontsize=25, prop={'size': 20, 'weight':'bold'})
plt.tight_layout()
# plt.savefig("cogsci_images/force_corr_PD.pdf")
# plt.savefig("cogsci_images/force_corr_PD.jpg")

In [ ]:
from isaac.visualization import make_frame_curried
import moviepy.editor as mpy

def make_clip(trial_data):

    duration = len(trial_data)

    n_bodies = sum(["o"+str(i)+".x" in list(trial_data.columns) for i in range(1, 5)])
    
    while (len(trial_data) + 1) % 60 != 0:
        trial_data = trial_data.append(trial_data.iloc[-1], ignore_index=True)
    make_frame = make_frame_curried(trial_data, n_bodies)
    clip = mpy.VideoClip(make_frame, duration=duration / 60)
    return clip, trial_data

replays = read_dataset("../new_exp_data/exp7_passive.h5")
clips = [make_clip(replay)[0] for replay in replays]

In [ ]:
def plot_information_curves_and_get_display(clips, correlation_dfs, question_type, idx):

    clips_and_corrs = sorted(zip(clips, correlation_dfs), key=lambda x: x[1][question_type+"_corr"].unique()[0], reverse=True)
    
    true_class = clips_and_corrs[idx][1].true_mass
    if question_type == "force":
        true_class = clips_and_corrs[idx][1].true_rel
    
    how = []
    for df in correlation_dfs:
        a = df["pd_"+question_type]
        b = df["model_correct_"+question_type+"_answer_probability"]
        how.append((a - b).mean())
        
    ws = 1
    f = plt.figure(figsize=(12, 6))
    ax = f.gca()

    clips_and_corrs[idx][1]["pd_"+question_type].rolling(window=ws).sum()[ws-1:].plot(label="PD "+question_type, linewidth=5, ax=ax)
    clips_and_corrs[idx][1]["model_correct_"+question_type+"_answer_probability"].rolling(window=ws).sum()[ws-1:].plot(label="RNN "+ question_type+" Int. Inf.", linewidth=5, ax=ax)
    print(true_class+ " " + str(clips_and_corrs[idx][1][question_type+"_corr"].unique()[0]))
    
    ax.legend(fontsize=20, prop={'size': 25, 'weight':'bold'})
    ax.set_xticks(np.arange(0, 46, 5))
    ax.set_xticklabels(np.arange(0, 46, 5), fontweight="bold")

    ax.set_yticks(np.arange(0, 11, 1) / 10)
    ax.set_yticklabels(np.arange(0, 11, 1) / 10, fontweight="bold")               

    ax.set_xlabel("second", fontsize=30, fontweight="bold")
    ax.grid()

    return clips_and_corrs[idx][0]
        # return 

In [ ]:
replay = plot_information_curves_and_get_display(clips, correlation_dfs, "force", 0)
plt.tick_params(axis="both", labelsize=25)
plt.tight_layout()
plt.savefig("cogsci_images/force_PD_RNN_good_example.pdf")
plt.savefig("cogsci_images/force_PD_RNN_good_example.jpg")
plt.show()
# replay.write_videofile("cogsci_images/force_PD_RNN_good_example.mp4", fps=24)

replay = plot_information_curves_and_get_display(clips, correlation_dfs, "force", -1)
plt.tick_params(axis="both", labelsize=25)
plt.tight_layout()
plt.savefig("cogsci_images/force_PD_RNN_bad_example.pdf")
plt.savefig("cogsci_images/force_PD_RNN_bad_example.jpg")
plt.show()
# replay.write_videofile("cogsci_images/force_PD_RNN_bad_example.mp4", fps=24)

replay = plot_information_curves_and_get_display(clips, correlation_dfs, "mass", 0)
plt.tick_params(axis="both", labelsize=25)
plt.tight_layout()
plt.savefig("cogsci_images/mass_PD_RNN_good_example.pdf")
plt.savefig("cogsci_images/mass_PD_RNN_good_example.jpg")
plt.show()
# replay.write_videofile("cogsci_images/mass_PD_RNN_good_example.mp4", fps=24)


replay = plot_information_curves_and_get_display(clips, correlation_dfs, "mass", -1)
plt.tick_params(axis="both", labelsize=25)
plt.tight_layout()
plt.savefig("cogsci_images/mass_PD_RNN_bad_example.pdf")
plt.savefig("cogsci_images/mass_PD_RNN_bad_example.jpg")
# replay.write_videofile("cogsci_images/mass_PD_RNN_bad_example.mp4", fps=24)

In [ ]:
nrows = 12
ncols = 36 // nrows

f, axis = plt.subplots(nrows, ncols, figsize=(25, 40))

question_type = "force"

correlation_dfs = sorted(correlation_dfs, key=lambda x: (x.true_rel, x[question_type+"_corr"].unique()[0]), reverse=True)

for i, df in enumerate(correlation_dfs):
    
    row = i // nrows
    col = i % nrows
    
    if df.is_force_correct:
        is_rnn_correct = "RNN is correct"
    else:
        is_rnn_correct = "RNN is wrong"
          
    title = "Corr: %.3f // %s" % (df.force_corr.unique()[0], is_rnn_correct)
    if col == 0:
        title = df.true_rel + "\n" + title

    axis[col][row].set_title(title, fontsize=25)
    df["pd_"+question_type].plot(ax=axis[col][row])
    df["model_correct_force_answer_probability"].plot(ax=axis[col][row])

plt.tight_layout()
plt.savefig("cogsci_images/all_PD_IER_force_plots.pdf")
plt.savefig("cogsci_images/all_PD_IER_force_plots.jpg")

In [ ]:
nrows = 12
ncols = 36 // nrows

f, axis = plt.subplots(nrows, ncols, figsize=(25, 40))

question_type = "mass"

correlation_dfs = sorted(correlation_dfs, key=lambda x: (x.true_mass, x[question_type+"_corr"].unique()[0]), reverse=True)


for i, df in enumerate(correlation_dfs):
        
    row = i // nrows
    col = i % nrows
    
    if df.is_mass_correct:
        is_rnn_correct = "RNN is correct"
    else:
        is_rnn_correct = "RNN is wrong"
          
    title = "Corr: %.3f // %s" % (df.mass_corr.unique()[0], is_rnn_correct)
    if col == 0:
        title = df.true_mass + "\n" + title

    axis[col][row].set_title(title, fontsize=25)
    df["pd_"+question_type].plot(ax=axis[col][row])
    df["model_correct_mass_answer_probability"].plot(ax=axis[col][row])

plt.tight_layout()
plt.savefig("cogsci_images/all_PD_IER_mass_plots.pdf")
plt.savefig("cogsci_images/all_PD_IER_mass_plots.jpg")